# pyAstroTrader

# Create Model

After downloading the quotes data for the asset selected with the ASSET_TO_CALCULATE environment variable, we need to add the astrological data to the quotes and then generate a XGBoost model

First of all, we need to import the models that we need to process.

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.metrics import mean_squared_error as mse

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import plot_tree

from IPython.display import display, HTML

import eli5



```pyastrotrader``` is a python module that we created, in order to calculate astrological charts based on specific dates, and also to calculate aspects between charts

In [ ]:
from pyastrotrader import calculate_chart, calculate_aspects, calculate_transits
from pyastrotrader.utils import create_input_json
from pyastrotrader.constants import *

Import all settings and helper functions that we will use in the next cells

In [ ]:
from settings import *
from helpers import *

Read the CSV file with the quotes downloaded, and also create a counter column to help in the calculated columns below

In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

Using several helper functions from ```helpers.py``` module, for each day we need to determine several indicators like:
* The current trend
* the future trend
* If there was a change in the trend ( a swing trade opportunity )
* current volatility for the previous days
* and many other indicators

In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )
StockPrices['PreviousStartDate'] = StockPrices.apply( lambda x :  get_previous_stock_date(StockPrices, x, SWING_TRADE_DURATION), axis =1 )
StockPrices['FutureFinalDate'] = StockPrices.apply( lambda x :  get_future_stock_date(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis =1 )

StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)

StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)
StockPrices['IntradayVolatility'] = StockPrices.apply(lambda x: calculate_intraday_volatility(StockPrices, x, SWING_TRADE_DURATION), axis =1)

StockPrices['FutureTrendMax'] = StockPrices.apply(lambda x : get_future_stock_max_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)
StockPrices['FutureTrendMin'] = StockPrices.apply(lambda x : get_future_stock_min_price(StockPrices, x, max_counter, SWING_TRADE_DURATION), axis = 1)

StockPrices['IsSwing'] = StockPrices.apply(lambda x: detect_swing_trade(x, SWING_EXPECTED_VOLATILITY), axis =1)
StockPrices['IsSwing'] = StockPrices.apply(lambda x:clean_swing_trade(StockPrices, x, SWING_EXPECTED_VOLATILITY), axis =1)

StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)
StockPrices['StockDecreasedPrice'] = StockPrices.apply(lambda x:detect_price_decrease(x, STAGNATION_THRESHOLD), axis =1)
StockPrices['StockStagnated'] = StockPrices.apply(lambda x:detect_price_stagnated(x, STAGNATION_THRESHOLD), axis =1)

StockPrices['TARGET_IS_SWING'] = StockPrices['IsSwing']
StockPrices['TARGET_PRICE_INCREASE'] = StockPrices['StockIncreasedPrice']
StockPrices['TARGET_PRICE_DECREASE'] = StockPrices['StockDecreasedPrice']
StockPrices['TARGET_PRICE_STAGNATION'] = StockPrices['StockStagnated']

After all the analisys, we can generate a excel file in order to help debug the indicators generated above

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

To debug the indicators, we can plot a stock chart with the swing indication, but this is commented out as it requires a lot of computational resources.

In [ ]:
"""
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['IsSwing'] == 1].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))
"""

In [ ]:
"""
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Detected Swing Trade Opportunities".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
#fig.show()
"""

Well, in order to calculate the astrological indicators for the current ASSET_TO_CALCULATE, we need to generate a natal chart of the asset, which traditionally is the first trade date on the current exchange 

In [ ]:
asset_natal_chart_input = create_input_json(NATAL_DATE, 
                                            DEFAULT_PARAMETERS, 
                                            DEFAULT_CONFIG)

asset_natal_chart = calculate_chart(asset_natal_chart_input)
dates_to_generate = list(StockPrices['CorrectedDate'])

Now, for all the dates on the pandas dataframe containing the quotes, we need to generate astrological charts with the list of planets to consider: ```PLANETS_TO_CALCULATE```, their aspects: ```ASPECTS_TO_CALCULATE```

In [ ]:
for current_date in dates_to_generate:
    chart_input = create_input_json(current_date + 'T10:00:00-03:00', 
                                      DEFAULT_PARAMETERS, 
                                      DEFAULT_CONFIG)
    charts[current_date] = calculate_chart(chart_input)
    aspects[current_date] = calculate_transits(asset_natal_chart, charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)
    aspects_transiting[current_date]= calculate_aspects(charts[current_date], PLANETS_TO_CALCULATE, ASPECTS_TO_CALCULATE, 4)   

We have the natal chart and also all the charts for each date in the pandas dataframe, now we need to add to the pandas dataframe, the astrological aspects that occur in each date, we will set only to 1 if there is a aspect occuring or 0 if not, we also will check for aspects on the transiting chart as well as aspects between the natal chart and the transiting chart

**astro_columns** will indicate the name of the columns containing astrological indicators in the pandas dataframe

In [ ]:
astro_columns = []

for first_planet in PLANETS_TO_CALCULATE:
    for second_planet in PLANETS_TO_CALCULATE:
        for aspect in ASPECTS_TO_CALCULATE:
            column_name="ASTRO_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices['StockIncreasedPrice'] = StockPrices.apply(lambda x:detect_price_increase(x, STAGNATION_THRESHOLD), axis =1)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')
            
            if first_planet == second_planet:
                continue
                
            column_name="ASTRO_TRANSITING_{}_{}_{}".format(PLANETS[first_planet],ASPECT_NAME[aspect],PLANETS[second_planet]).upper()
            astro_columns.append(column_name)
            StockPrices[column_name] = StockPrices.apply(lambda x:is_aspected_transiting(x, first_planet, second_planet, aspect), axis =1)
            StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float', errors='coerce')                 

We need also to determine which planets are retrograde in each date of the pandas dataframe

In [ ]:
for first_planet in PLANETS_TO_CALCULATE:
    column_name="ASTRO_{}_RETROGADE".format(PLANETS[first_planet]).upper()
    astro_columns.append(column_name)
    StockPrices[column_name] = StockPrices.apply(lambda x:is_retrograde(x, first_planet), axis =1)
    StockPrices[column_name] = pd.to_numeric(StockPrices[column_name],  downcast='float',errors='coerce')

After the pandas dataframe has been populated with the astrological indicators, we can now train the XGBoost models in order to predict the following target variables:
* Price Increase: There is a increase in price after that date
* Price Decrease: There is a decrease in price after that date
* Price Stagnation: There is a stagnation in price after that date
* Swing Trade: There is a change in trend after that date
    
**Important to notice that we will use as input only the astro_columns columns which contains astrological indicators**

In [ ]:
booster_price_increase, score_price_increase = get_best_booster('TARGET_PRICE_INCREASE', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Increase Model:{}".format(score_price_increase))

booster_price_decrease, score_price_decrease = get_best_booster('TARGET_PRICE_DECREASE', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Decrease Model:{}".format(score_price_decrease))

booster_price_stagnation, score_price_stagnation = get_best_booster('TARGET_PRICE_STAGNATION', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Price Stagnation Model:{}".format(score_price_stagnation))

booster_swing_trade, score_swing_trade = get_best_booster('TARGET_IS_SWING', MAX_INTERACTIONS, StockPrices, astro_columns)
print("Best Score for Swing Trade Model:{}".format(score_swing_trade))

We can now save each model score in a text file

In [ ]:
score_price_increase_file_name = './output/{}.score.price.increase.txt'.format(ASSET_TO_CALCULATE)
score_price_decrease_file_name = './output/{}.score.price.decrease.txt'.format(ASSET_TO_CALCULATE)
score_price_stagnation_file_name = './output/{}.score.price.stagnation.txt'.format(ASSET_TO_CALCULATE)
score_swing_trade_file_name = './output/{}.score.swing.trade.txt'.format(ASSET_TO_CALCULATE)

with open(score_price_increase_file_name, 'w') as f:
    f.write(str(score_price_increase))

with open(score_price_decrease_file_name, 'w') as f:
    f.write(str(score_price_decrease))

with open(score_price_stagnation_file_name, 'w') as f:
    f.write(str(score_price_stagnation))

with open(score_swing_trade_file_name, 'w') as f:
    f.write(str(score_swing_trade))

We can also calculate for each model the relevant astrological variables, used in each model

In [ ]:
relevant_features_swing_trade = sorted( ((v,k) for k,v in booster_swing_trade.get_score().items()), reverse=True)
relevant_features_price_increase = sorted( ((v,k) for k,v in booster_price_increase.get_score().items()), reverse=True)
relevant_features_price_decrease = sorted( ((v,k) for k,v in booster_price_decrease.get_score().items()), reverse=True)
relevant_features_price_stagnation = sorted( ((v,k) for k,v in booster_price_stagnation.get_score().items()), reverse=True)

display(relevant_features_swing_trade)
display(relevant_features_price_increase)

We can now write such features to text files in order to improve the analisys of the model.

In [ ]:
def write_features(f, list_to_write):
    for item_to_write in list_to_write:
        f.write(str(item_to_write).replace(')','').replace('(','').replace('\'','').replace(' ','') + '\n')
        
features_price_increase_file_name = './output/{}.features.price.increase.txt'.format(ASSET_TO_CALCULATE)
features_price_decrease_file_name = './output/{}.features.price.decrease.txt'.format(ASSET_TO_CALCULATE)
features_price_stagnation_file_name = './output/{}.features.price.stagnation.txt'.format(ASSET_TO_CALCULATE)
features_swing_trade_file_name = './output/{}.features.swing.trade.txt'.format(ASSET_TO_CALCULATE)

with open(features_price_increase_file_name, 'w') as f:
    write_features(f,relevant_features_price_increase)

with open(features_price_decrease_file_name, 'w') as f:
    write_features(f,relevant_features_price_decrease)

with open(features_price_stagnation_file_name, 'w') as f:
    write_features(f,relevant_features_price_stagnation)

with open(features_swing_trade_file_name, 'w') as f:
    write_features(f,relevant_features_swing_trade)

We can check the predicted value for each model on the pandas dataframe, creating a column for it 

In [ ]:
StockPrices['PredictSwingTradeScore'] = StockPrices.apply(lambda x:predict_score(x, booster_swing_trade, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceIncreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_increase, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceDecreaseScore'] = StockPrices.apply(lambda x:predict_score(x, booster_price_decrease, StockPrices, astro_columns), axis =1)
StockPrices['PredictPriceStagnation'] = StockPrices.apply(lambda x:predict_score(x, booster_price_stagnation, StockPrices, astro_columns), axis =1)

And save the model for further use on the ```Predict.ipynb``` notebook

In [ ]:
booster_swing_trade.save_model('./output/{}_swing_trade.model'.format(ASSET_TO_CALCULATE))
booster_price_increase.save_model('./output/{}_price_increase.model'.format(ASSET_TO_CALCULATE))
booster_price_decrease.save_model('./output/{}_price_decrease.model'.format(ASSET_TO_CALCULATE))
booster_price_stagnation.save_model('./output/{}_price_stagnation.model'.format(ASSET_TO_CALCULATE))

And save a excel with all the data produced...

In [ ]:
output_excel_file='./output/{}.Analisys.xlsx'.format(ASSET_TO_CALCULATE)
StockPrices.to_excel(output_excel_file)

The plotting of charts has been commented out as it is very resource consuming...

In [ ]:
"""
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictSwingTradeScore'] > 0.9].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))
"""        

In [ ]:
"""
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Swing Trade Opportunities detected by XGBoost".format(ASSET_TO_CALCULATE),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
#fig.show()
"""

In [ ]:
"""
swing_to_chart = []
for index, current_swing in StockPrices[StockPrices['PredictPriceIncreaseScore'] > 5].iterrows():
    swing_to_chart.append(dict(
        x0=current_swing['CorrectedDate'], 
        x1=current_swing['CorrectedDate'], 
        y0=0, 
        y1=1, 
        xref='x', 
        yref='paper',
        line_width=2))
"""        

In [ ]:
"""
fig = go.Figure(data=[go.Candlestick(
                x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(
    title="{} Price Increase Opportunities detected by XGBoost (Min {}%)".format(ASSET_TO_CALCULATE, STAGNATION_THRESHOLD),
    width=1000,
    height=500,
    xaxis_rangeslider_visible=False,
    shapes=swing_to_chart,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=30,
        pad=4
    ),    
)
#fig.show()
"""